In [ ]:
# !pip install llama_index
# !pip install langchain
# !pip install openai

In [1]:
from llama_index import (
    GPTVectorStoreIndex,
    ResponseSynthesizer,
    download_loader
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor
import os
import openai
import json


youtubelink = 'https://www.youtube.com/watch?v=FQBou-YgxyE'

def open_file(filepath):
    with open(filepath, 'r') as infile:
        return infile.read()

class Chatbot:
    def __init__(self, api_key, index):
        self.index = index
        openai.api_key = api_key
        self.chat_history = []

    def generate_response(self, user_input):
        prompt = "\n".join([f"{message['role']}: {message['content']}" for message in self.chat_history[-5:]])
        prompt += f"\nUser: {user_input}"
        query_engine = index.as_query_engine()
        response = query_engine.query(user_input)
        # response = index.query(user_input)

        message = {"role": "assistant", "content": response.response}
        self.chat_history.append({"role": "user", "content": user_input})
        self.chat_history.append(message)
        return message
    
    def load_chat_history(self, filename):
        try:
            with open(filename, 'r') as f:
                self.chat_history = json.load(f)
        except FileNotFoundError:
            pass

    def save_chat_history(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.chat_history, f)

openai.api_key = open_file('./data/openaiapikey.txt')          
os.environ['OPENAI_API_KEY'] = openai.api_key

YoutubeTranscriptReader = download_loader("YoutubeTranscriptReader")

loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=[youtubelink])
index = GPTVectorStoreIndex.from_documents(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 7274 tokens


In [2]:
import textwrap  # Import the textwrap module

# Swap out your index below for whatever knowledge base you want
bot = Chatbot(openai.api_key, index=index)
bot.load_chat_history("chat_history.json")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        print("Bot: Goodbye!")
        bot.save_chat_history("chat_history.json")
        break
    response = bot.generate_response(user_input)
    # print(f"Bot: {response['content']}")
    # Wrap the text and print the wrapped content
    wrapped_content = textwrap.wrap(response['content'], width=90)
    print("User: ", user_input)
    print("Bot: ",)
    for line in wrapped_content:
        print(line)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 9 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2042 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  what is the advantage of using List index?
Bot: 
 The advantage of using a List index is that it allows you to explicitly go through every
single piece of context within the index and use every single piece of context within the
index to synthesize and answer for a given task. This is especially useful for
summarization tasks, where you need to go through an entire document or set of documents
to generate a summary. List index also allows you to store relationships between nodes,
which can be used to generate more complex answers.


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 9 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1991 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  what is the disadvantage of using list index?
Bot: 
 One disadvantage of using list index is that it can be difficult to understand whether or
not the knowledge has been included or not, especially for users without a lot of ML or
deep learning experience. Additionally, it can be expensive if the data volumes are large.


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2141 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  tell more about the Vector Store index,  pros and cons
Bot: 
 The Vector Store index is a data structure that allows for the storage of nodes and
associated embeddings in a vector database. This index is useful for retrieving the most
similar nodes to a query, as the query embedding can be used to retrieve the top K nodes
from the vector database.   The pros of the Vector Store index include the ability to
quickly retrieve the most similar nodes to a query, as well as the ability to store nodes
and associated embeddings in a vector database. Additionally, the Vector Store index
integrates with a variety of downstream vector databases, such as Chroma Pinecone, BVA
Quadrant, and more.   The cons of the Vector Store index include the fact that it can be
more expensive and have higher latency than other data structures, as it requires the
retrieval of all nodes in the list. Additionally, the Vector Store index does not support
multimodal documents, such as images.


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2002 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  which index can best work for summarization type of work?
Bot: 
 The List Index is best suited for summarization type of work. It allows for explicit
traversal of the data and synthesis of a final answer. It can also be used to compose a
graph of index structures over the data, allowing for more complex queries to be answered.
Additionally, it can be used to convert natural language queries into SQL queries that can
be executed against a SQL database.


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1749 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


User:  how the multi-step query works 
Bot: 
 Multi-step queries work by breaking a complex query into multiple simpler ones over the
same data source. This allows the user to ask all the questions possible over the same
data source to get back the answer. For example, if you wanted to know who was in the
first batch of an accelerator program the author started, you could break this overall
question down into simpler sub questions that you can answer sequentially until you have
enough information to generate a final answer. The first question might be "what
accelerator program did the author start?" and then you can ask "who is in the first batch
of that accelerator program?" over the same data source. This effectively is similar to
train of thought prompting but with the exception that it is specifically over your data
source.
Bot: Goodbye!
